# Madlibs

## Madlibs style word substitution
Given a (multiline) f-string, match the substitutes to the targets. If the targets have no subs, put an empty string (default value).

In [4]:
# Attempt one
def g(inp:str, **subs) -> str:
    return inp.format(**subs)

In [5]:
replacements = {"x":70}
# replacements = {"x":70, "w":80, "z": 90}
# replacements = {"x":70, "w":80, }

sample = "The number is {x} {w} {z}"
try:
    test = g(sample, **replacements)
    print(test)
except KeyError as e:
    print(e)

'w'


Problem: if the dict doesnt contain it, theres a keyerror

In [6]:
# Attempt 2: use a custom formatter (PEP 3101)
from string import Formatter
from typing import Dict
class MadLibber(Formatter):
    def __init__(self, default="") -> None:
        super().__init__()
        self.default=default

    def get_value(self, key, args, kwds:Dict):
        if isinstance(key, str):
            return kwds.get(key, self.default)
        else:
            return super().get_value(key, args, kwds)


In [7]:
mL = MadLibber()
print(mL.format(sample, **replacements))

The number is 70  


Hopefully we can improve on its inelegance. Also I dont fully understand the formatter class.

In [8]:
# Attempt 3: format_map
class Default(dict):
    def __missing__(self, key):
        return '{'+key+'}'
class Default2(dict):
    def __missing__(self, key):
        return ""

In [9]:
print(sample.format_map(Default(replacements)))
print(sample.format_map(Default2(replacements)))

The number is 70 {w} {z}
The number is 70  


In [10]:
multiline ="""
Lorem Ipsum is {adj} dummy text of the printing 
and typesetting industry. Lorem Ipsum has been the 
industry's standard dummy text ever since the 1500s, 
when an unknown printer took a {noun} of type and 
scrambled it to make a type specimen book. It has 
survived not only five centuries, but also the 
leap {preposition} electronic typesetting, remaining 
essentially unchanged. It was {verb} in the 
1960s with the release of Letraset sheets containing 
Lorem Ipsum passages, and more recently with 
desktop publishing software like Aldus PageMaker 
including versions of Lorem Ipsum.
"""
subs = {"adj":"interestingly",
        "noun": "human",
        "preposition": "at"}
print(multiline.format_map(Default(subs)))
print(multiline.format_map(Default2(subs)))


Lorem Ipsum is interestingly dummy text of the printing 
and typesetting industry. Lorem Ipsum has been the 
industry's standard dummy text ever since the 1500s, 
when an unknown printer took a human of type and 
scrambled it to make a type specimen book. It has 
survived not only five centuries, but also the 
leap at electronic typesetting, remaining 
essentially unchanged. It was {verb} in the 
1960s with the release of Letraset sheets containing 
Lorem Ipsum passages, and more recently with 
desktop publishing software like Aldus PageMaker 
including versions of Lorem Ipsum.


Lorem Ipsum is interestingly dummy text of the printing 
and typesetting industry. Lorem Ipsum has been the 
industry's standard dummy text ever since the 1500s, 
when an unknown printer took a human of type and 
scrambled it to make a type specimen book. It has 
survived not only five centuries, but also the 
leap at electronic typesetting, remaining 
essentially unchanged. It was  in the 
1960s with the rel

## Getting a corpus of passages
Let's actually make a game! We want lots of different passages of similar lengths.

In [11]:
import pandas as pd

In [12]:
# extract articles from https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail?resource=download
df = pd.read_csv("archive/cnn_dailymail/test.csv")
articles_df = df['article'].str.replace('. ','.\n', regex=False) #

In [13]:
# check the first article in a single file
with open("passage.txt", "w") as f:
    text = articles_df.iloc[0]
    print(text)
    f.write(text)
    

Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk.
They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger.
More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans.
'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for space 

In [14]:
# export to CSV
articles_df.to_csv("articles.csv")


## NLP
We want to use part-of-speech taggers to tag words to different "parts of speech": nouns, verbs, adjectives, adverbs, connectives, pronouns and prepositions, etc.
This allows us to choose certain numbers of each word type to blank out.

Let's try out spaCy on just the first article using this [guide](https://freecontent.manning.com/detecting-word-types-with-part-of-speech-tagging-part-1/).

Note: We also have to run `py -m  spacy download en_core_web_sm` to download that model before we can use it. We're just using the small model for testing purposes.

In [15]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc= nlp(text)

Let's look inside them using the inspect module.

In [16]:
import inspect
print(type(doc))
print(inspect.getmembers(doc[0]))

<class 'spacy.tokens.doc.Doc'>
[('_', <spacy.tokens.underscore.Underscore object at 0x00000258851355D0>), ('__bytes__', <built-in method __bytes__ of spacy.tokens.token.Token object at 0x0000025886E51FD0>), ('__class__', <class 'spacy.tokens.token.Token'>), ('__delattr__', <method-wrapper '__delattr__' of spacy.tokens.token.Token object at 0x0000025886E51FD0>), ('__dir__', <built-in method __dir__ of spacy.tokens.token.Token object at 0x0000025886E51FD0>), ('__doc__', 'An individual token – i.e. a word, punctuation symbol, whitespace,\n    etc.\n\n    DOCS: https://spacy.io/api/token\n    '), ('__eq__', <method-wrapper '__eq__' of spacy.tokens.token.Token object at 0x0000025886E51FD0>), ('__format__', <built-in method __format__ of spacy.tokens.token.Token object at 0x0000025886E51FD0>), ('__ge__', <method-wrapper '__ge__' of spacy.tokens.token.Token object at 0x0000025886E51FD0>), ('__getattribute__', <method-wrapper '__getattribute__' of spacy.tokens.token.Token object at 0x000002588

That's not very useful. Let's try a different approach.

In [17]:
token = doc[0]
print([token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop])

['Ever', 'ever', 'ADV', 'RB', 'advmod', 'Xxxx', True, True]


In [18]:
for token in doc:
    print([token.text, token.pos_, spacy.explain(token.pos_)]) # .pos is an int, .pos_ is a string

['Ever', 'ADV', 'adverb']
['noticed', 'VERB', 'verb']
['how', 'SCONJ', 'subordinating conjunction']
['plane', 'NOUN', 'noun']
['seats', 'NOUN', 'noun']
['appear', 'VERB', 'verb']
['to', 'PART', 'particle']
['be', 'AUX', 'auxiliary']
['getting', 'VERB', 'verb']
['smaller', 'ADJ', 'adjective']
['and', 'CCONJ', 'coordinating conjunction']
['smaller', 'ADJ', 'adjective']
['?', 'PUNCT', 'punctuation']
['With', 'ADP', 'adposition']
['increasing', 'VERB', 'verb']
['numbers', 'NOUN', 'noun']
['of', 'ADP', 'adposition']
['people', 'NOUN', 'noun']
['taking', 'VERB', 'verb']
['to', 'ADP', 'adposition']
['the', 'DET', 'determiner']
['skies', 'NOUN', 'noun']
[',', 'PUNCT', 'punctuation']
['some', 'DET', 'determiner']
['experts', 'NOUN', 'noun']
['are', 'AUX', 'auxiliary']
['questioning', 'VERB', 'verb']
['if', 'SCONJ', 'subordinating conjunction']
['having', 'VERB', 'verb']
['such', 'ADJ', 'adjective']
['packed', 'VERB', 'verb']
['out', 'ADP', 'adposition']
['planes', 'NOUN', 'noun']
['is', 'AUX', 

Alright, we've got POS tagging done! Now to put take out certain parts of the text.

We use spacy's matcher to look for words with certain POS tags, such as verbs, and replace them with a placeholder or suggestion.

In [27]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern = [
    {
        "POS" :"VERB", # match verbs
    "OP":"*"           # match 0 or more verbs
    }
    ]
matcher.add("verb", [pattern])
matches = matcher(doc)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

6360137228241296794 verb 1 2 noticed
6360137228241296794 verb 5 6 appear
6360137228241296794 verb 8 9 getting
6360137228241296794 verb 14 15 increasing
6360137228241296794 verb 18 19 taking
6360137228241296794 verb 26 27 questioning
6360137228241296794 verb 28 29 having
6360137228241296794 verb 30 31 packed
6360137228241296794 verb 34 35 putting
6360137228241296794 verb 41 42 say
6360137228241296794 verb 44 45 shrinking
6360137228241296794 verb 55 56 putting
6360137228241296794 verb 66 67 squabbling
6360137228241296794 verb 72 73 shrinking
6360137228241296794 verb 76 77 putting
6360137228241296794 verb 92 93 set
6360137228241296794 verb 99 100 said
6360137228241296794 verb 111 112 set
6360137228241296794 verb 115 116 flying
6360137228241296794 verb 122 123 stipulate
6360137228241296794 verb 138 139 have
6360137228241296794 verb 149 150 said
6360137228241296794 verb 169 170 take
6360137228241296794 verb 181 182 crowding
6360137228241296794 verb 190 191 fighting
6360137228241296794 verb 

Note: A [Span](https://spacy.io/api/span) is a slice from the document.

In [51]:
def replace_word(orig_doc):
    text = ''
    buffer_start = 0
    for _, match_start, _ in matcher(orig_doc):
        if match_start > buffer_start:  # If we've skipped over some tokens, let's add those in (with trailing whitespace if available)
            text += orig_doc[buffer_start: match_start].text + orig_doc[match_start - 1].whitespace_
            
        token = orig_doc[match_start]
        tense = token.morph.get('Tense')[0] if token.morph.get('Tense') else ""
        long_pos = spacy.explain(token.pos_)
        tag = spacy.explain(token.tag_)
        replacement = f"<{tag}>"
        text += replacement + token.whitespace_  # Replace token, with trailing whitespace if available
        buffer_start = match_start + 1
    text += orig_doc[buffer_start:].text
    return text

replace_word(doc)

"Ever <verb, past tense> how plane seats <verb, non-3rd person singular present> to be <verb, gerund or present participle> smaller and smaller? With <verb, gerund or present participle> numbers of people <verb, gerund or present participle> to the skies, some experts are <verb, gerund or present participle> if <verb, gerund or present participle> such <verb, past participle> out planes is <verb, gerund or present participle> passengers at risk.\nThey <verb, non-3rd person singular present> that the <verb, gerund or present participle> space on aeroplanes is not only uncomfortable - it's <verb, gerund or present participle> our health and safety in danger.\nMore than <verb, gerund or present participle> over the arm rest, <verb, gerund or present participle> space on planes <verb, gerund or present participle> our health and safety in danger? This week, a U.S consumer advisory group <verb, past participle> up by the Department of Transportation <verb, past tense> at a public hearing th

We've replace all verbs in a given text with a suggestion (that's a tad verbose). Let's try finding the number of each POS in a text, as both absolute number and percentage of the word count.

Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk.
They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger.
More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans.
'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for space 